In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import matplotlib.pyplot as plt
import re
import string
import nltk
import seaborn as sns 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from bs4 import BeautifulSoup
from nltk.tokenize import RegexpTokenizer

In [2]:
train=pd.read_csv('train.csv')
train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [3]:
sent_4000=train['tweet'].values[4000]
print(sent_4000)

Happy Founders Day #holiday #iphone #smile #shades #glasses #gh #ghana #chilling http://instagram.com/p/tQHyJlOK5G/


In [4]:
train.shape

(7920, 3)

In [5]:
#converting into lower case
train['tidy_tweet']=train['tweet'].apply(lambda x:x.lower())

In [6]:
train.head()

,id,label,tweet,tidy_tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,#fingerprint #pregnancy test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...,finally a transparant silicon case ^^ thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...,we love this! would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...,i'm wired i know i'm george i was made that wa...
4,5,1,What amazing service! Apple won't even talk to...,what amazing service! apple won't even talk to...


In [7]:
#Removing the html tags
def remove_html_tags(text):
    soup=BeautifulSoup(text,'lxml')
    html_free=soup.get_text()
    return html_free

In [8]:
#Removing @users
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt    

In [9]:
train['tidy_tweet']=np.vectorize(remove_pattern)(train['tidy_tweet'],'@[\w]*')

In [10]:
sent_207=train['tidy_tweet'].values[205:250]
print(sent_207)

["too bad  doesn't carry any #lumia devices but at least i was able to pick up a #samsung ativ s neo #windowsphone"
 'we would like to wish you an amazing day! make every minute count #tls #today #iphone #accessories #news #life july 10, 2017 at 07:00…'
 'my ipads new home :) #ipad #leopardprint #neon #ipad3 #apple http://instagram.com/p/hqkb2ssujl/'
 'my #iphone #apps i can listen to drive time hip hop from nyc whilst peeling potatoes in my kitchen!!'
 '#dissatisfied #apple #iphone6splus not for runners, it has issues with filter in camera sweat through #incasearmband'
 'he loves the fuck outta me.  #$&*# #imessage #iphone #text #gay #hesa$&*# http://instagram.com/p/aitqo5r6la/'
 'she is as ready as i am to do this! #chromebook #samsung #hellokitty #diamond #shine #ready http://instagram.com/p/jnmhgynfq_/'
 'beach group text. #beach #ocean #cellphone #friends #trio #iphone  karebear5891 http://instagram.com/p/fndf5bp6lm/'
 'at last i have my #apple watch 42mm space black stainless ste

In [11]:
def cleaning(text):

    import string
    exclude = set(string.punctuation)

    import re
    # remove new line and digits with regular expression
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\d', '', text)
    # remove patterns matching url format
    url_pattern = r'((http|ftp|https):\/\/)?[\w\-_]+(\.[\w\-_]+)+([\w\-\.,@?^=%&amp;:/~\+#]*[\w\-\@?^=%&amp;/~\+#])?'
    text = re.sub(url_pattern, ' ', text)
    # remove non-ascii characters
    text = ''.join(character for character in text if ord(character) < 128)
    # remove punctuations
    text = ''.join(character for character in text if character not in exclude)
    # standardize white space
    text = re.sub(r'\s+', ' ', text)
    # drop capitalization
    text = text.lower()
    #remove white space
    text = text.strip()

    return text

In [12]:
train['tidy_tweet']=train['tidy_tweet'].apply(lambda x : cleaning(x))

In [13]:
train['tidy_tweet'].values[205:250]

array(['too bad doesnt carry any lumia devices but at least i was able to pick up a samsung ativ s neo windowsphone',
       'we would like to wish you an amazing day make every minute count tls today iphone accessories news life july at',
       'my ipads new home ipad leopardprint neon ipad apple',
       'my iphone apps i can listen to drive time hip hop from nyc whilst peeling potatoes in my kitchen',
       'dissatisfied apple iphonesplus not for runners it has issues with filter in camera sweat through incasearmband',
       'he loves the fuck outta me imessage iphone text gay hesa',
       'she is as ready as i am to do this chromebook samsung hellokitty diamond shine ready',
       'beach group text beach ocean cellphone friends trio iphone karebear',
       'at last i have my apple watch mm space black stainless steel excited overthemoon',
       'i l ny hug follow like iphone tattoo christmas beautiful newyork',
       'id totally fuck siri her knowledge excites me siri fucki

In [15]:
#removing the short word of len>2
train['tidy_tweet']=train['tidy_tweet'].apply(lambda x:' '.join([w for w in x.split() if len(w)>2]))

In [16]:
train['tidy_tweet'].values[205:250]

array(['too bad doesnt carry any lumia devices but least was able pick samsung ativ neo windowsphone',
       'would like wish you amazing day make every minute count tls today iphone accessories news life july',
       'ipads new home ipad leopardprint neon ipad apple',
       'iphone apps can listen drive time hip hop from nyc whilst peeling potatoes kitchen',
       'dissatisfied apple iphonesplus not for runners has issues with filter camera sweat through incasearmband',
       'loves the fuck outta imessage iphone text gay hesa',
       'she ready this chromebook samsung hellokitty diamond shine ready',
       'beach group text beach ocean cellphone friends trio iphone karebear',
       'last have apple watch space black stainless steel excited overthemoon',
       'hug follow like iphone tattoo christmas beautiful newyork',
       'totally fuck siri her knowledge excites siri fuckinsiri',
       'lol love iphone rtto apple who fuckedup the iphone know what about get yes pow via',

In [17]:
#tokenizing
tokenizer=RegexpTokenizer(r'\w+')

In [18]:
train['tidy_tweet']=train['tidy_tweet'].apply(lambda x:tokenizer.tokenize(x.lower()))

In [19]:
train['tidy_tweet'].values[205:215]

array([list(['too', 'bad', 'doesnt', 'carry', 'any', 'lumia', 'devices', 'but', 'least', 'was', 'able', 'pick', 'samsung', 'ativ', 'neo', 'windowsphone']),
       list(['would', 'like', 'wish', 'you', 'amazing', 'day', 'make', 'every', 'minute', 'count', 'tls', 'today', 'iphone', 'accessories', 'news', 'life', 'july']),
       list(['ipads', 'new', 'home', 'ipad', 'leopardprint', 'neon', 'ipad', 'apple']),
       list(['iphone', 'apps', 'can', 'listen', 'drive', 'time', 'hip', 'hop', 'from', 'nyc', 'whilst', 'peeling', 'potatoes', 'kitchen']),
       list(['dissatisfied', 'apple', 'iphonesplus', 'not', 'for', 'runners', 'has', 'issues', 'with', 'filter', 'camera', 'sweat', 'through', 'incasearmband']),
       list(['loves', 'the', 'fuck', 'outta', 'imessage', 'iphone', 'text', 'gay', 'hesa']),
       list(['she', 'ready', 'this', 'chromebook', 'samsung', 'hellokitty', 'diamond', 'shine', 'ready']),
       list(['beach', 'group', 'text', 'beach', 'ocean', 'cellphone', 'friends', 'trio',

In [20]:
#removing stop words
words=stopwords.words('english')

In [21]:
def remove_stopwords(text):
    word=[w for w in text if w not in words]
    return word

In [22]:
train['tidy_tweet']=train['tidy_tweet'].apply(lambda x:remove_stopwords(x))

In [23]:
train['tidy_tweet'].values[205:230]

array([list(['bad', 'doesnt', 'carry', 'lumia', 'devices', 'least', 'able', 'pick', 'samsung', 'ativ', 'neo', 'windowsphone']),
       list(['would', 'like', 'wish', 'amazing', 'day', 'make', 'every', 'minute', 'count', 'tls', 'today', 'iphone', 'accessories', 'news', 'life', 'july']),
       list(['ipads', 'new', 'home', 'ipad', 'leopardprint', 'neon', 'ipad', 'apple']),
       list(['iphone', 'apps', 'listen', 'drive', 'time', 'hip', 'hop', 'nyc', 'whilst', 'peeling', 'potatoes', 'kitchen']),
       list(['dissatisfied', 'apple', 'iphonesplus', 'runners', 'issues', 'filter', 'camera', 'sweat', 'incasearmband']),
       list(['loves', 'fuck', 'outta', 'imessage', 'iphone', 'text', 'gay', 'hesa']),
       list(['ready', 'chromebook', 'samsung', 'hellokitty', 'diamond', 'shine', 'ready']),
       list(['beach', 'group', 'text', 'beach', 'ocean', 'cellphone', 'friends', 'trio', 'iphone', 'karebear']),
       list(['last', 'apple', 'watch', 'space', 'black', 'stainless', 'steel', 'excited

In [24]:
lemmi=WordNetLemmatizer()
def lemmitizing(text):
    lemm_text=[lemmi.lemmatize(i) for i in text]
    return lemm_text

In [27]:
train['tidy_tweet']=train['tidy_tweet'].apply(lambda x:lemmitizing(x))

In [28]:
train['tidy_tweet'].values[205:215]

array([list(['bad', 'doesnt', 'carry', 'lumia', 'device', 'least', 'able', 'pick', 'samsung', 'ativ', 'neo', 'windowsphone']),
       list(['would', 'like', 'wish', 'amazing', 'day', 'make', 'every', 'minute', 'count', 'tl', 'today', 'iphone', 'accessory', 'news', 'life', 'july']),
       list(['ipads', 'new', 'home', 'ipad', 'leopardprint', 'neon', 'ipad', 'apple']),
       list(['iphone', 'apps', 'listen', 'drive', 'time', 'hip', 'hop', 'nyc', 'whilst', 'peeling', 'potato', 'kitchen']),
       list(['dissatisfied', 'apple', 'iphonesplus', 'runner', 'issue', 'filter', 'camera', 'sweat', 'incasearmband']),
       list(['love', 'fuck', 'outta', 'imessage', 'iphone', 'text', 'gay', 'hesa']),
       list(['ready', 'chromebook', 'samsung', 'hellokitty', 'diamond', 'shine', 'ready']),
       list(['beach', 'group', 'text', 'beach', 'ocean', 'cellphone', 'friend', 'trio', 'iphone', 'karebear']),
       list(['last', 'apple', 'watch', 'space', 'black', 'stainless', 'steel', 'excited', 'overth

In [31]:
stemmer=PorterStemmer()
def stemming_text(text):
    stem_text=' '.join([stemmer.stem(i) for i in text])
    return stem_text

In [32]:
train['tidy_tweet']=train['tidy_tweet'].apply(lambda x:stemming_text(x))

In [33]:
train['tidy_tweet'].values[205:215]

array(['bad doesnt carri lumia devic least abl pick samsung ativ neo windowsphon',
       'would like wish amaz day make everi minut count tl today iphon accessori news life juli',
       'ipad new home ipad leopardprint neon ipad appl',
       'iphon app listen drive time hip hop nyc whilst peel potato kitchen',
       'dissatisfi appl iphonesplu runner issu filter camera sweat incasearmband',
       'love fuck outta imessag iphon text gay hesa',
       'readi chromebook samsung hellokitti diamond shine readi',
       'beach group text beach ocean cellphon friend trio iphon karebear',
       'last appl watch space black stainless steel excit overthemoon',
       'hug follow like iphon tattoo christma beauti newyork'],
      dtype=object)